In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../dataset/wk9/outcome_data.csv')

target_string = 'Responded'

df_X = df.iloc[:, :-1]
df_y = df.iloc[:, -1]

xx = df_X[df_X['District'] == 'Suburban']

df_y[xx.index]




0     Not responded
1     Not responded
7     Not responded
8         Responded
10        Responded
Name: Outcome, dtype: object

In [4]:
class Node:

    def __init__(self):
        self.branch = []
        self.children = []
        self.feature = ''
        self.level = 0
        self.is_leaf = False
        self.result = None


In [5]:
def get_impurity(target):
    hit = np.where(target == target_string)[0]
    flag = len(hit) == 0 or len(hit) == len(target)

    return flag, len(hit)

In [6]:
def find_info_gain(train, target, column):

    target_idx = np.where(target == target_string)[0]
    target_count = len(target_idx)

    elems = train[column].unique()
    total_ratio = target_count / train.shape[0]
    entropy_total = - (total_ratio * np.log2(total_ratio) + (1 - total_ratio) * np.log2(1 - total_ratio))
    entropys = []

    for elem in elems:
        entropy = 0

        X_index =  train[train[column] == elem].index
        y_index = target[target == target_string].index

        intersec = np.intersect1d(X_index, y_index)

        if len(intersec) == len(X_index) or len(intersec) == 0:
            entropy = 0

        else:
            ratio = len(intersec) / len(X_index)
            entropy = - (ratio * np.log2(ratio) + (1 - ratio) * np.log2(1 - ratio))
            entropy *= len(intersec)

        entropys.append(entropy)


    ret = entropy_total - sum(entropys) / target_count
    return ret

In [29]:
def grow_tree(node, X, y, level):

    print(X)
    print(y)

    flag, hit_cnt = get_impurity(y)
    if flag:
        node.level = level
        node.is_leaf = True
        node.result = target_string if hit_cnt != 0 else "Not respond"
        return

    columns = X.columns
    entropys = []
    for col in columns:
        entropys.append(find_info_gain(X, y, col))

    idx = np.argmax(entropys)

    selected_feature = columns[idx]
    node.feature = selected_feature
    node.level = level

    categories = X[selected_feature].unique()

    for category in categories:
        filtered_X = X[X[selected_feature] == category]
        filtered_X_idx = filtered_X.index
        filtered_y = y[filtered_X_idx]
        filtered_X.drop(columns = [selected_feature], inplace=True)

        child = Node()
        node.branch.append(child)
        node.children.append(category)
        grow_tree(child, filtered_X, filtered_y, level + 1)



root = Node()
grow_tree(root, df_X, df_y, 0)



    District     House Type Income Previous Customer
0   Suburban       Detached   High                No
1   Suburban       Detached   High               Yes
2      Rural       Detached   High                No
3      Urban  Semi-detached   High                No
4      Urban  Semi-detached    Low                No
5      Urban  Semi-detached    Low               Yes
6      Rural  Semi-detached    Low               Yes
7   Suburban        Terrace   High                No
8   Suburban  Semi-detached    Low                No
9      Urban        Terrace    Low                No
10  Suburban        Terrace    Low               Yes
11     Rural        Terrace   High               Yes
12     Rural       Detached    Low                No
13     Urban        Terrace   High               Yes
0     Not responded
1     Not responded
2         Responded
3         Responded
4         Responded
5     Not responded
6         Responded
7     Not responded
8         Responded
9         Responded
10   

/opt/homebrew/Caskroom/miniforge/base/envs/RL_Sutton/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/homebrew/Caskroom/miniforge/base/envs/RL_Sutton/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/homebrew/Caskroom/miniforge/base/envs/RL_Sutton/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [30]:
def print_info(node, width, rule):
    """
    Method to print the infromation about the tree
    """
    # Defining the number of spaces
    const = int(node.level * width ** 1.5)
    spaces = "-" * const

    if node.level == 0:
        print("Root")

    if node.is_leaf:
        print(f"|{spaces} Prediction: is {node.result}")

    print(f"|{spaces} Split rule: {node.feature} is {rule}")
    print(f"{' ' * const}   | level: {node.level}")

def print_tree(node):
    """
    Prints the whole tree from the current node to the bottom
    """
    print_info(node, 4, node.children[idx])

    children = node.branch

    for idx, child in enumerate(children):
        print_info(child, 4, node.children[idx])




In [31]:
print_tree(root)

Root
| Split rule: District is ['Suburban', 'Rural', 'Urban']
   | level: 0
|-------- Split rule: Income is Suburban
           | level: 1
|-------- Prediction: is Responded
|-------- Split rule:  is Rural
           | level: 1
|-------- Split rule: Previous Customer is Urban
           | level: 1
